In [15]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
# Load CSV files from sources
    # oly_athletes = https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results
    # world_pop = https://data.worldbank.org/indicator/SP.POP.TOTL
    # gdp = https://knoema.com/tlcnrcg/gdp-by-country-world-largest-economies-2016

oly_athletes = pd.read_csv('olympics/athlete_events.csv')
world_pop = pd.read_csv('WorldPopulationData.csv')
gdp = pd.read_csv('GDPcountry20142016.csv')

In [31]:
# Original Olympics data
oly_athletes.head()

,Name,Sex,Country,Code,Year,Season,City,Sport,Medal
0,A Dijiang,M,China,CHN,1992,Summer,Barcelona,Basketball,NaN
1,A Lamusi,M,China,CHN,2012,Summer,London,Judo,NaN
2,Gunnar Nielsen Aaby,M,Denmark,DEN,1920,Summer,Antwerpen,Football,NaN
3,Edgar Lindenau Aabye,M,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Gold
4,Christine Jacoba Aaftink,F,Netherlands,NED,1988,Winter,Calgary,Speed Skating,NaN


In [6]:
# Cleaning Olympics Data
# Subset columns that we needed to examine
oly_athletes = oly_athletes[['Name', 'Sex', 'Team', 'NOC','Year', 'Season', 'City', 'Sport', 'Medal']]
oly_athletes.columns=['Name', 'Sex', 'Country', 'Code','Year', 'Season', 'City', 'Sport', 'Medal']
oly_athletes.shape

(271116, 9)

In [33]:
# Original World Population data
world_pop.head()

,Country,Code,2014,2016
0,Aruba,ABW,103774.0,104872.0
1,Afghanistan,AFG,33370794.0,35383128.0
2,Angola,AGO,26941779.0,28842484.0
3,Albania,ALB,2889104.0,2876101.0
4,Andorra,AND,79213.0,77297.0


In [8]:
# Cleaning World Population Data
# Subset and rename columns for Olympic years
world_pop = world_pop[['Country Name', 'Country Code', '2014', '2016']]
world_pop.columns = ['Country', 'Code', '2014', '2016']
world_pop.head()

,Country,Code,2014,2016
0,Aruba,ABW,103774.0,104872.0
1,Afghanistan,AFG,33370794.0,35383128.0
2,Angola,AGO,26941779.0,28842484.0
3,Albania,ALB,2889104.0,2876101.0
4,Andorra,AND,79213.0,77297.0


In [9]:
# Original GDP Data
gdp.head()

,Country,2014,2015,2016
0,United States,17521.75,18219.30,18707.15
1,China,10534.53,11226.19,11221.84
2,Japan,4850.41,4389.48,4926.67
3,Germany,3904.92,3383.09,3496.61
4,United Kingdom,3036.31,2897.06,2669.11


In [10]:
# Cleaning GDP Data
# Subset and rename columns for Olympic years
gdp = gdp[['Country', '2014', '2016']]
gdp.head()

,Country,2014,2016
0,United States,17521.75,18707.15
1,China,10534.53,11221.84
2,Japan,4850.41,4926.67
3,Germany,3904.92,3496.61
4,United Kingdom,3036.31,2669.11


In [27]:
# Set up Database connection
# Using postgres relational database
rds_connection_string = "postgres:MomentoMori314?@localhost:5432/ETL-Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [28]:
# Create table for Olympics data
oly_athletes.to_sql('oly_athletes', engine)

In [29]:
# Create table for GDP data
gdp.to_sql('gdp', engine)

In [30]:
# Create table for World Population data
world_pop.to_sql('world_pop', engine)

In [34]:
pd.read_sql_query('select * from oly_athletes', con=engine).head()

,index,Name,Sex,Country,Code,Year,Season,City,Sport,Medal
0,0,A Dijiang,M,China,CHN,1992,Summer,Barcelona,Basketball,None
1,1,A Lamusi,M,China,CHN,2012,Summer,London,Judo,None
2,2,Gunnar Nielsen Aaby,M,Denmark,DEN,1920,Summer,Antwerpen,Football,None
3,3,Edgar Lindenau Aabye,M,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Gold
4,4,Christine Jacoba Aaftink,F,Netherlands,NED,1988,Winter,Calgary,Speed Skating,None


In [35]:
pd.read_sql_query('select * from gdp', con=engine).head()

,index,Country,2014,2016
0,0,United States,17521.75,18707.15
1,1,China,10534.53,11221.84
2,2,Japan,4850.41,4926.67
3,3,Germany,3904.92,3496.61
4,4,United Kingdom,3036.31,2669.11


In [36]:
pd.read_sql_query('select * from world_pop', con=engine).head()

,index,Country,Code,2014,2016
0,0,Aruba,ABW,103774.0,104872.0
1,1,Afghanistan,AFG,33370794.0,35383128.0
2,2,Angola,AGO,26941779.0,28842484.0
3,3,Albania,ALB,2889104.0,2876101.0
4,4,Andorra,AND,79213.0,77297.0


In [ ]:
# once we had the data pulled into PostGres, 
# we had to create the PKs and FKs 
# as that was not transfered over when creating the tables through sqlalchemy 
# we added them and then had to troubleshoot some of that data by adding information into the tables.